<a href="https://colab.research.google.com/github/jimmy-io/Coursera_Capstone/blob/master/LA_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Capstone Project

## Jimmy J.

### Introduction
Frank Lloyd Wright said it best - “Tip the world over on its side and everything loose will land in Los Angeles." For decades people from all over the world have come to this city in search of home, family, love, fame, and fortune. Los Angeles welcomes this multicultural migration with a suburban sprawl that encompases over 88 cities, and even more unincorporated neihborhoods. Each of these neighborhoods are characterized by geographical, economic, and cultural features that make them uniquely poised to cater to different demographics.
Neighborhoods like Downtown Los Angeles, Culver City, Long Beach, Century City, and West Hollywood provide a mixture of urban style living and accessibility to grocery stores, malls, means of public transportation and entertainment venues within walking distance. On the otherhand suburbs like Baldwin Hills, Crenshaw, Echo Park and Boyle Heights are quiter neighborhoods with single family dwellings that are not easily accessible via public modes of transportation.
Given the vast spectrum of possibilites of neighborhoods to choose from in LA, someone looking to move here might be overwhelmed. In this project I've attempted to characterize neighborhoods in LA by the nature of venue that are in their immediate vicinities using a clustering algorithm. Results from this analysis show that neighborhoods in LA fall under a few groups, defined by the nature of the venues closest to them. This is of most interest to rental unit searching apps like Westiside Rentals or Rentpad, to real estate agents, and generally, to people looking to move to LA. The results of this project can help them find neighborhoods that are most aligned with what they are looking for in a place to live and overall, provide a more satisfactory experience than chosing a neighborhood at random.

### Data
The list of neighborhoods in Los Angeles was web scraped from Wikipedia using BeautifulSoup. The names of these neigborhoods were then fed into Nominatim to obtain their geographical coordinates.
The venues in the vicinity of these neighborhoods will be retreived using the FourSquare API search engine. Venues of five different categories were chosen: Travel & Transport, Arts & Entertainment, Outdoors & Recreation, Nightlife Spot, and Food. The number of venues of each category for each neighborhood was counted and then normalized to give five parameters with which to cluster the neighborhoods by.

In [0]:
### Importing libraries 

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import re
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import bs4 as bs
import urllib.request
import requests
import collections
import csv

In [2]:
#@hidden_cell

CLIENT_ID = 'JS0P2BHNS4GICN4OT1LRM03JV0OLTO4QWS0I5AEITRLVI3QU' # your Foursquare ID
CLIENT_SECRET = 'KGFP21SEFHLUXM2EPAI4HDLQOAI21MC1CY24RJ4AII4UX2Q3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails: hidden')

Your credentails: hidden


In [31]:
# Retreiving the coordinates for Los Angeles 

address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="LA_explorer")
location = geolocator.geocode(address)
LAlatitude = location.latitude
LAlongitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(LAlatitude, LAlongitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [12]:
## Categories and IDs from FourSquare API

catIds={}
catIds={
        'Food':'4d4b7105d754a06374d81259',
 
       }
catIds

{'Food': '4d4b7105d754a06374d81259'}

In [33]:
for key in catIds:
    print(key)
    print(catIds[key])

Food
4d4b7105d754a06374d81259


In [0]:
### Foursqaure API
venues_list=[]
LIMIT = 10000 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
categoryId='4d4b7105d754a06374d81259'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(CLIENT_ID, 
                                                                                                                                              CLIENT_SECRET, 
                                                                                                                                              VERSION, 
                                                                                                                                              LAlatitude,
                                                                                                                                              LAlongitude, 
                                                                                                                                              radius,
                                                                                                                                              categoryId,
                                                                                                                                              LIMIT)
            
    # make the GET request
            
results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
    # return only relevant information for each nearby venue
venues_list.append([(v['venue']['name'], 
                      v['venue']['location']['lat'], 
                      v['venue']['location']['lng'],  
                      v['venue']['categories'][0]['name']) for v in results])
            
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Venue', 
                          'Venue Latitude', 
                          'Venue Longitude', 
                          'Venue Category']
        



In [48]:
nearby_venues

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Redbird,34.050666,-118.244068,American Restaurant
1,Marugame Monzo,34.049807,-118.240202,Udon Restaurant
2,KazuNori: The Original Hand Roll Bar,34.047716,-118.247452,Sushi Restaurant
3,Sari Sari Store LA,34.051065,-118.249390,Filipino Restaurant
4,Café Dulcé,34.048869,-118.240508,Bakery
...,...,...,...,...
95,Kang Ho Dong Baek Jeong,34.063828,-118.297364,Korean Restaurant
96,Ham Ji Park,34.063633,-118.295838,Korean Restaurant
97,California Donuts,34.068965,-118.293116,Donut Shop
98,Night + Market Song,34.086948,-118.275748,Thai Restaurant


In [18]:
venues_dict['Food'].shape

(3310, 7)

In [0]:
## Saving the nearby venues data as a csv to avoid repeated API calls 

for key in venues_dict:
    key1=re.sub('[^A-Za-z0-9&]+ ', '', key)
    key1=re.sub('\W+','', key1)
    venues_dict[key].to_csv('/content/drive/My Drive/IBM Capstone/'+'LA_'+str(key1)+".csv")

In [0]:
## Reading the nearby venues data as a csv
LA_VENUES={}
for key in catIds:
    key1=re.sub('[^A-Za-z0-9&]+ ', '', key)
    key1=re.sub('\W+','', key1)
    LA_VENUES[key]=pd.read_csv('/content/drive/My Drive/IBM Capstone/'+'LA_'+str(key1)+'.csv')
    LA_VENUES[key].dropna(inplace = True)
    LA_VENUES[key].drop(columns = ['Unnamed: 0'], inplace = True)

In [14]:
for key in LA_VENUES:
    print(key)
    print(LA_VENUES[key].shape)

Food
(3310, 7)


In [15]:
LA_VENUES['Food'].head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandie,34.015951,-118.283513,Figueroa Philly Cheese Steak,34.014254,-118.282527,Sandwich Place
1,Adams-Normandie,34.015951,-118.283513,Chick-fil-A,34.016633,-118.282575,Fast Food Restaurant
2,Adams-Normandie,34.015951,-118.283513,Chipotle Mexican Grill,34.016956,-118.282584,Mexican Restaurant
3,Adams-Normandie,34.015951,-118.283513,Pizza Studio,34.018569,-118.281668,Pizza Place
4,Adams-Normandie,34.015951,-118.283513,Holbox,34.017515,-118.278439,Seafood Restaurant


In [0]:
def get_superset_of_column_names_from_file(json_file_path):
    """Read in the json dataset file and return the superset of column names."""
    column_names = set()
    with open(json_file_path) as fin:
        for line in fin:
            line_contents = json.loads(line)
            column_names.update(
                    set(get_column_names(line_contents).keys())
                    )
    return column_names
def get_column_names(line_contents, parent_key=''):
    """Return a list of flattened key names given a dict.
    Example:
        line_contents = {
            'a': {
                'b': 2,
                'c': 3,
                },
        }
        will return: ['a.b', 'a.c']
    These will be the column names for the eventual csv file.
    """
    column_names = []
    for k, v in line_contents.items():
        column_name = "{0}.{1}".format(parent_key, k) if parent_key else k
        if isinstance(v, collections.MutableMapping):
            column_names.extend(
                    get_column_names(v, column_name).items()
                    )
        else:
            column_names.append((column_name, v))
    return dict(column_names)

column_names = get_superset_of_column_names_from_file('/content/drive/My Drive/IBM Capstone/yelp_academic_dataset_business.json')

In [66]:
 def read_and_write_file(json_file_path, csv_file_path, column_names):
    """Read in the json dataset file and write it out to a csv file, given the column names."""
    with open(csv_file_path, 'w') as fout:
        csv_file = csv.writer(fout)
        csv_file.writerow(list(column_names))
        with open(json_file_path) as fin:
            for line in fin:
                line_contents = json.loads(line)
                csv_file.writerow(get_row(line_contents, column_names))

 read_and_write_file('/content/drive/My Drive/IBM Capstone/yelp_academic_dataset_business.json' , '/content/drive/My Drive/IBM Capstone/yelp_academic_dataset_business.csv' , column_names)

TypeError: ignored